In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from matplotlib.pyplot import table
%matplotlib inline

In [ ]:
df = pd.read_csv(r'C:\PythonProject\churn-analysis\.venv\data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

In [ ]:
df.info()


In [ ]:
plt.hist(df['MonthlyCharges'])
plt.show()

In [ ]:
plt.hist(df['MonthlyCharges'],edgecolor='k')
plt.title('Распределение месячной платы за услуги')
plt.xlabel('плата в месяц, $')
plt.ylabel('кол-во наблюдений')
plt.grid()

In [ ]:
pic = sns.histplot(data=df,x='MonthlyCharges')
pic.set(xlabel ="плата в месяц, $", ylabel = "кол-во наблюдений", title = 'распределение меячной платы за услуги')
plt.grid()

In [ ]:
pic = sns.histplot(data=df,x='MonthlyCharges', hue = 'SeniorCitizen')
pic.set(xlabel ="плата в месяц, $", ylabel = "кол-во наблюдений", title = 'распределение меячной платы за услуги')
plt.grid()

In [ ]:
hist = go.Histogram(x=df['MonthlyCharges'], nbinsx=20)
fig = go.Figure(data=hist)

fig.update_layout(
    width=600,
    height=400,
    title='Распределение платы за услуги компании',
    xaxis_title='Плата за услуги',
    yaxis_title='Кол-во наблюдений'
)
fig.show()

In [ ]:
df.hist()

In [ ]:
df['PaymentMethod'].value_counts()

In [ ]:
sns.countplot(y=df['PaymentMethod'])

In [ ]:
df['PaymentMethod'].value_counts().head(3).index

In [ ]:
sns.countplot(df[df['PaymentMethod'].isin(df['PaymentMethod'].value_counts().head(3).index)]['PaymentMethod'])

In [ ]:
df1=df.groupby('tenure').agg({'TotalCharges':'mean'}).reset_index()
df1.head()

In [ ]:
line = go.Scatter(x=df1['tenure'], y=df1['TotalCharges'], mode='lines')
fig=go.Figure(data=line)

fig.update_layout(
    autosize=False,
    width=700,
    height=400,
    title='Зависимость средни выплат клиента от времени жизни с компанией',
    xaxis_title='месяцев с подкл услугами',
    yaxis_title='средние выплаты клиента'
)

In [ ]:
plt.scatter(df['MonthlyCharges'],df['tenure']);
plt.xlabel('месячная плата')
plt.ylabel('Срок жизни клиента в компании, мес')

In [ ]:
df['Churn'].map({'No':'blue', 'Yes':'orange'}).head()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df['MonthlyCharges'],df['tenure'],
            color=df['Churn'].map({'No':'blue', 'Yes':'red'}),
            alpha=0.6)
plt.title('Распределение срока жизни клиента и средн платы по тарифу')
plt.xlabel('Месячная плата по тарифу')
plt.ylabel('Срок жизни клиента в компании')
plt.show()

In [ ]:
df[df['Churn']=='Yes'].head()

In [ ]:
sns.boxplot(x='Churn',y='MonthlyCharges',data=df)


In [ ]:
df['churn_flag']=df['Churn'].map({'Yes':1,'No':0})

In [ ]:
table=pd.pivot_table(data=df,index='InternetService',columns='Contract',values='churn_flag',aggfunc='mean')
table

In [ ]:
sns.heatmap(table)

In [ ]:
sns.barplot(data=df, x='SeniorCitizen', y='churn_flag', estimator='mean', errorbar=None, palette='Blues')

# Оформление
plt.title('Доля ушедших клиентов (Churn) по статусу пенсионера')
plt.xlabel('SeniorCitizen (0 = Нет, 1 = Да)')
plt.ylabel('Доля ушедших клиентов')
plt.xticks([0, 1], ['Не пенсионер', 'Пенсионер'])
plt.show()

In [ ]:
sns.catplot(data=df, x='SeniorCitizen', hue='Churn', kind='count', palette='Set1')
plt.title('Количество ушедших и оставшихся по статусу пенсионера')
plt.xticks([0, 1], ['Не пенсионер', 'Пенсионер'])
plt.show()

In [ ]:
services = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Заменяем 'Yes' → 1, остальное (включая 'No internet service') → 0
df['NumServices'] = df[services].applymap(lambda x: 1 if x == 'Yes' else 0).sum(axis=1)


In [ ]:
df_internet = df[df['InternetService'] != 'No'].copy()

In [ ]:
df_internet['churn_flag'] = df_internet['Churn'].map({'Yes': 1, 'No': 0})

# Группируем по типу интернета и числу услуг
grouped = df_internet.groupby(['InternetService', 'NumServices'], as_index=False)['churn_flag'].mean()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=grouped,
    x='NumServices',
    y='churn_flag',
    hue='InternetService',
    marker='o',
    linewidth=2.5,
    palette='Set1'
)

plt.title('Доля оттока в зависимости от количества подключённых услуг\n(среди клиентов с интернетом)', fontsize=14)
plt.xlabel('Количество подключённых услуг')
plt.ylabel('Доля ушедших клиентов (Churn Rate)')
plt.xticks(range(0, 7))
plt.grid(alpha=0.3)
plt.legend(title='Тип интернета')
plt.show()

In [ ]:
# Фильтр: только пенсионеры
seniors = df[df['SeniorCitizen'] == 1]

# Доля ушедших (Churn == 'Yes')
churn_rate_seniors = (seniors['Churn'] == 'Yes').mean()

# В процентах, округлённо до целых
churn_rate_seniors_percent = round(churn_rate_seniors * 100)
print(churn_rate_seniors_percent)

In [ ]:
senior_customers = df[df['SeniorCitizen'] == 1]

# Считаем долю ушедших (Churn == 'Yes')
churn_rate = (senior_customers['Churn'] == 'Yes').mean()

# Переводим в проценты и округляем до целого
churn_rate_percent = round(churn_rate * 100)

print(churn_rate_percent)